In [1]:
train_data_dir = "D:/python/projects/cars and trees detection new/modified_dataset"
train_voc_label_dir = "D:/python/projects/cars and trees detection new/modified_dataset"
labels = {"background": 0, "tree": 1, "car": 2}

In [2]:
from torch.utils.data import Dataset
import os
import xml.etree.ElementTree as ET

class LampDataset(Dataset):
    def __init__(self, data_dir, label_dir, transform=None):
        self.data_dir = data_dir
        self.label_dir = label_dir
        self.transform = transform
        self.labels = self.load_labels()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_path = self.labels[idx]['image_path']
        label_path = self.labels[idx]['label_path']

        image = Image.open(image_path).convert('RGB')
        label = self.load_label(label_path)

        image = self.transform(image)

        return image, label

    def load_labels(self):
        labels = []
        for filename in os.listdir(self.label_dir):
            if filename.endswith('.xml'):
                label_path = os.path.join(self.label_dir, filename)
                image_path = os.path.join(self.data_dir, os.path.splitext(filename)[0] + '.jpg')
                labels.append({'image_path': image_path, 'label_path': label_path})
        return labels

    def load_label(self, label_path):
        tree = ET.parse(label_path)
        root = tree.getroot()
        
        filename = root.find('filename').text
        
        boxes = []
        names = []
        for obj in root.findall('object'):
            names.append(labels[obj.find('name').text])
                
            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])

        label = {'boxes': torch.tensor(boxes, dtype=torch.float32),
                 'labels': torch.tensor(names, dtype=torch.int64),
                 'filename': filename}
        return label

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image

transform = transforms.Compose([
    transforms.ToTensor(), # Convert the image to a tensor
])

# create dataset objects for training and validation
train_dataset = LampDataset(train_data_dir, train_voc_label_dir, transform=transform)

In [4]:
import torchvision
import torch.optim as optim
import time
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=3)

#coco_names = ['lamp']

#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
#model.roi_heads.box_predictor.cls_score = nn.Linear(1024,len(coco_names))

#model = Network()
model = model.to(device)

dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)

D:\programs\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\programs\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import torch.optim as optim
import time
import torch.nn.functional as F

def train(model):
    # Defining the optimizer
    optimizer = optim.SGD(model.parameters(),lr = 0.1)

    num_of_epochs = 50
    
    if not os.path.exists('models'):
        os.mkdir('models')

    for epoch in range(num_of_epochs):

        model.train()
        
        for batch, (x, y) in enumerate(dataloader):
            # Converting data from cpu to GPU if available to improve speed
            # x = image, y = labels, z = boxes
            boxes = y['boxes']
            labels = y['labels']
            
            x,boxes,labels = x.to(device),boxes.to(device),labels.to(device)            

            targets=[{'boxes': boxes, 'labels': labels}]
            
            targets = [{'boxes': b, 'labels': l} for b, l in zip(targets[0]['boxes'], targets[0]['labels'])]

            
            print("FILENAME: ", y['filename'])
            print("target: ", targets)
            
            loss_dict = model(x, targets)
           
            losses = sum(loss for loss in loss_dict.values())

            print("losses: ", losses)
        
            optimizer.zero_grad()
                    
            losses.backward()
            
            optimizer.step()

            print("BATCH #: ", batch)
        
        print("EPOCH #: ", epoch)
            

In [6]:
train(model)

FILENAME:  ['tashkent_108.jpg']
target:  [{'boxes': tensor([[ 64., 238., 322., 370.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(1.2524, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  0
FILENAME:  ['tashkent_39.jpg']
target:  [{'boxes': tensor([[239., 171., 331., 342.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.3761, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  1
FILENAME:  ['stirling_16.jpg']
target:  [{'boxes': tensor([[  2.,   2., 145., 318.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.4418, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  2
FILENAME:  ['tashkent_78.jpg']
target:  [{'boxes': tensor([[210., 123., 298., 312.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.1713, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  3
FILENAME:  ['tashkent_36.jpg']
target:  [{'boxes': tensor([[239., 235., 307., 287.]], device='cuda:0'), 'labels

FILENAME:  ['tashkent_49.jpg']
target:  [{'boxes': tensor([[ 99., 185., 199., 267.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.1587, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  38
FILENAME:  ['tashkent_14.jpg']
target:  [{'boxes': tensor([[ 55.,  48., 186., 402.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0858, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  39
FILENAME:  ['tashkent_19.jpg']
target:  [{'boxes': tensor([[340., 251., 416., 468.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.1303, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  40
FILENAME:  ['tashkent_55.jpg']
target:  [{'boxes': tensor([[208., 217., 315., 260.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.2307, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  41
FILENAME:  ['tashkent_100.jpg']
target:  [{'boxes': tensor([[284., 389., 416., 550.]], device='cuda:0'), 'la

losses:  tensor(0.1187, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  75
FILENAME:  ['tashkent_74.jpg']
target:  [{'boxes': tensor([[  3.,   3., 327., 455.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.1690, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  76
FILENAME:  ['stirling_22.jpg']
target:  [{'boxes': tensor([[  3., 210., 148., 312.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.1484, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  77
FILENAME:  ['stirling_19.jpg']
target:  [{'boxes': tensor([[172.,  21., 306., 182.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.3426, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  78
FILENAME:  ['tashkent_24.jpg']
target:  [{'boxes': tensor([[ 55.,  36., 143., 169.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.2322, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  79
FILENAME:  ['tashkent_28.jpg']

FILENAME:  ['tashkent_46.jpg']
target:  [{'boxes': tensor([[ 60., 143., 214., 215.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.1307, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  113
FILENAME:  ['tashkent_25.jpg']
target:  [{'boxes': tensor([[245., 202., 371., 362.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.2374, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  114
FILENAME:  ['stirling_10.jpg']
target:  [{'boxes': tensor([[205.,   5., 409., 211.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.2389, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  115
FILENAME:  ['tashkent_17.jpg']
target:  [{'boxes': tensor([[ 32.,  70., 416., 443.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.2091, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  116
FILENAME:  ['tashkent_33.jpg']
target:  [{'boxes': tensor([[206., 268., 335., 362.]], device='cuda:0'), 

FILENAME:  ['tashkent_14.jpg']
target:  [{'boxes': tensor([[ 55.,  48., 186., 402.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0747, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  20
FILENAME:  ['tashkent_55.jpg']
target:  [{'boxes': tensor([[208., 217., 315., 260.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.1119, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  21
FILENAME:  ['tashkent_81.jpg']
target:  [{'boxes': tensor([[  1., 153.,  68., 340.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.2911, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  22
FILENAME:  ['tashkent_51.jpg']
target:  [{'boxes': tensor([[ 42., 209., 114., 272.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.1140, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  23
FILENAME:  ['stirling_12.jpg']
target:  [{'boxes': tensor([[133.,   0., 360., 203.]], device='cuda:0'), 'lab

...

FILENAME:  ['tashkent_56.jpg']
target:  [{'boxes': tensor([[122., 160., 251., 223.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0094, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  86
FILENAME:  ['tashkent_32.jpg']
target:  [{'boxes': tensor([[ 99., 172., 329., 434.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0170, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  87
FILENAME:  ['tashkent_105.jpg']
target:  [{'boxes': tensor([[351., 346., 408., 394.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0065, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  88
FILENAME:  ['stirling_12.jpg']
target:  [{'boxes': tensor([[133.,   0., 360., 203.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0114, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  89
FILENAME:  ['tashkent_69.jpg']
target:  [{'boxes': tensor([[154., 309., 393., 476.]], device='cuda:0'), 'la

FILENAME:  ['stirling_5.jpg']
target:  [{'boxes': tensor([[ 59.,  68., 175., 234.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0579, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  123
FILENAME:  ['stirling_0.jpg']
target:  [{'boxes': tensor([[163., 171., 254., 294.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0295, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  124
FILENAME:  ['tashkent_64.jpg']
target:  [{'boxes': tensor([[239.,   0., 296., 262.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0184, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  125
FILENAME:  ['tashkent_97.jpg']
target:  [{'boxes': tensor([[126., 414., 192., 467.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0065, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  126
FILENAME:  ['tashkent_37.jpg']
target:  [{'boxes': tensor([[ 48., 234., 122., 283.]], device='cuda:0'), 'l

FILENAME:  ['tashkent_29.jpg']
target:  [{'boxes': tensor([[310.,  12., 351., 591.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0785, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  30
FILENAME:  ['stirling_3.jpg']
target:  [{'boxes': tensor([[259.,  97., 290., 140.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0181, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  31
FILENAME:  ['tashkent_100.jpg']
target:  [{'boxes': tensor([[284., 389., 416., 550.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0163, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  32
FILENAME:  ['tashkent_108.jpg']
target:  [{'boxes': tensor([[ 64., 238., 322., 370.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0257, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  33
FILENAME:  ['tashkent_67.jpg']
target:  [{'boxes': tensor([[103., 267., 232., 328.]], device='cuda:0'), 'la

losses:  tensor(0.0238, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  67
FILENAME:  ['tashkent_106.jpg']
target:  [{'boxes': tensor([[319., 455., 379., 484.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0632, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  68
FILENAME:  ['tashkent_39.jpg']
target:  [{'boxes': tensor([[239., 171., 331., 342.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0190, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  69
FILENAME:  ['tashkent_76.jpg']
target:  [{'boxes': tensor([[149., 291., 244., 334.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0204, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  70
FILENAME:  ['tashkent_77.jpg']
target:  [{'boxes': tensor([[  0.,   0., 356., 517.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0099, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  71
FILENAME:  ['tashkent_45.jpg'

FILENAME:  ['tashkent_95.jpg']
target:  [{'boxes': tensor([[ 28., 368., 404., 534.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0076, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  105
FILENAME:  ['tashkent_28.jpg']
target:  [{'boxes': tensor([[ 76.,   3., 286., 457.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0')}]
losses:  tensor(0.0160, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  106
FILENAME:  ['tashkent_0.jpg']
target:  [{'boxes': tensor([[191., 243., 263., 290.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0136, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  107
FILENAME:  ['tashkent_15.jpg']
target:  [{'boxes': tensor([[  0., 254., 363., 393.]], device='cuda:0'), 'labels': tensor([2], device='cuda:0')}]
losses:  tensor(0.0081, device='cuda:0', grad_fn=<AddBackward0>)
BATCH #:  108
FILENAME:  ['stirling_13.jpg']
target:  [{'boxes': tensor([[ 41.,   0., 394., 270.]], device='cuda:0'), '